# WideDeep_FTTransformer

## 라이브러리 설치

In [ ]:
# !pip -q install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124

In [4]:
# !pip install pytorch-widedeep

In [ ]:
# !pip install opencv-python-headless -> 저는 이거 없으면 엘리스에서 에러 나서 깔았습니다.

## import

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import shutil
import datetime
import os
import warnings
import random
warnings.simplefilter(action='ignore', category=FutureWarning)


from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, FunctionTransformer, QuantileTransformer, MultiLabelBinarizer
from sklearn.impute import SimpleImputer



import torch

# pytorch-widedeep 라이브러리 import
from pytorch_widedeep import Trainer
from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
from pytorch_widedeep.models import Wide, FTTransformer, WideDeep
from pytorch_widedeep.metrics import Accuracy,F1Score
from pytorch_widedeep.callbacks import EarlyStopping, ModelCheckpoint





## Data Load

In [ ]:
train_path = f'../data/train.csv'
test_path = f'../data/test.csv'
sample_path = f'../data/sample_submission.csv'


train = pd.read_csv(train_path).drop(columns=["ID"])
test = pd.read_csv(test_path).drop(columns=["ID"])
print(train.shape, test.shape)

(205080, 68) (51271, 67)


## Data Pre-processing

In [8]:
def drop_columns(df):
    cols = [
        '불임 원인 - 여성 요인',  # 고유값 1
        '불임 원인 - 정자 면역학적 요인',  # train, test 모두 '1'인 데이터 1개 >> 신뢰할 수 없음
        '난자 해동 경과일',
    ]
    df = df.drop(cols, axis=1)
    return df

def 특정시술유형(train, test):
    def categorize_procedure(proc):
        tokens = [token.strip() for token in proc.split(",") if token.strip() and not token.strip().isdigit()]
        # 우선순위에 따른 범주화
        if tokens.count("Unknown") >= 1:
            return "Unknown"
        if tokens.count("AH") >= 1:
            return "AH"
        if tokens.count("BLASTOCYST") >= 1:
            return "BLASTOCYST"
        if tokens.count("ICSI") >= 2 or tokens.count("IVF") >= 2:
            return "2ICSI_2IVF"
        if tokens.count("IVF") >= 1 and tokens.count("ICSI") >= 1:
            return "IVF_ICSI"
        if tokens == "ICSI":
            return "ICSI"
        if tokens == "IVF":
            return "IVF"
        return ",".join(tokens) if tokens else None

    for df in [train, test]:
        df['특정 시술 유형'] = df['특정 시술 유형'].str.replace(" / ", ",")
        df['특정 시술 유형'] = df['특정 시술 유형'].str.replace(":", ",")
        df['특정 시술 유형'] = df['특정 시술 유형'].str.replace(" ", "")

    counts = train['특정 시술 유형'].value_counts()
    allowed_categories = counts[counts >= 100].index.tolist()

    # allowed_categories에 속하지 않는 값은 "Unknown"으로 대체
    train.loc[~train['특정 시술 유형'].isin(allowed_categories), '특정 시술 유형'] = "Unknown"
    test.loc[~test['특정 시술 유형'].isin(allowed_categories), '특정 시술 유형'] = "Unknown"

    train['특정 시술 유형'] = train['특정 시술 유형'].apply(categorize_procedure)
    test['특정 시술 유형'] = test['특정 시술 유형'].apply(categorize_procedure)

    train['시술유형_통합'] = train['시술 유형'].astype(str) + '_' + train['특정 시술 유형'].astype(str)
    test['시술유형_통합'] = test['시술 유형'].astype(str) + '_' + test['특정 시술 유형'].astype(str)

    drop_cols = ['시술 유형', '특정 시술 유형']
    train = train.drop(drop_cols, axis=1)
    test = test.drop(drop_cols, axis=1)

    return train, test

def 시술횟수(df_train):
    for col in [col for col in df_train.columns if '횟수' in col]:
        df_train[col] = df_train[col].replace({'6회 이상':'6회'})
        df_train[col] = df_train[col].str[0].astype(int)
    df_train['시술_임신'] = df_train['총 임신 횟수'] - df_train['총 시술 횟수']
    df_train = df_train.drop('총 시술 횟수', axis=1)
    return df_train

def 배란유도유형(df_train, df_test):
    mapping = {
        '기록되지 않은 시행': 1,
        '알 수 없음': 0,
        '세트로타이드 (억제제)': 0,
        '생식선 자극 호르몬': 0,
    }
    df_train['배란 유도 유형'] = df_train['배란 유도 유형'].replace(mapping)
    df_test['배란 유도 유형'] = df_test['배란 유도 유형'].replace(mapping)
    return df_train, df_test

def 난자기증자나이(df_train, df_test):
    mapping = {
        '만20세 이하': 20,
        '만21-25세': 25,
        '만26-30세': 30,
        '만31-35세': 35,
        '알 수 없음': 20,  # 만20세 이하와 동일하게 처리
    }
    df_train['난자 기증자 나이'] = df_train['난자 기증자 나이'].replace(mapping)
    df_test['난자 기증자 나이'] = df_test['난자 기증자 나이'].replace(mapping)
    return df_train, df_test

def 배아생성주요이유(df_train, df_test):
    df_train['배아 생성 주요 이유'] = df_train['배아 생성 주요 이유'].fillna('DI')
    df_test['배아 생성 주요 이유'] = df_test['배아 생성 주요 이유'].fillna('DI')

    df_train['배아 생성 이유 리스트'] = df_train['배아 생성 주요 이유'].apply(lambda x: [reason.strip() for reason in x.split(',')])
    df_test['배아 생성 이유 리스트'] = df_test['배아 생성 주요 이유'].apply(lambda x: [reason.strip() for reason in x.split(',')])

    mlb = MultiLabelBinarizer()
    train_one_hot = pd.DataFrame(
        mlb.fit_transform(df_train['배아 생성 이유 리스트']),
        columns=mlb.classes_,
        index=df_train.index
    )
    train_one_hot.columns = ['배아생성이유_' + col for col in train_one_hot.columns]

    test_one_hot = pd.DataFrame(
        mlb.transform(df_test['배아 생성 이유 리스트']),
        columns=mlb.classes_,
        index=df_test.index
    )
    test_one_hot.columns = ['배아생성이유_' + col for col in test_one_hot.columns]

    df_train = pd.concat([df_train, train_one_hot], axis=1)
    df_test = pd.concat([df_test, test_one_hot], axis=1)

    cols_to_drop = [
        '배아 생성 주요 이유',
        '배아 생성 이유 리스트',
        '배아생성이유_연구용',
        '배아생성이유_DI'
    ]
    df_train = df_train.drop(cols_to_drop, axis=1, errors='ignore')
    df_test = df_test.drop(cols_to_drop, axis=1, errors='ignore')

    cols = ['배아생성이유_기증용',
            '배아생성이유_난자 저장용',
            '배아생성이유_배아 저장용',
            '배아생성이유_현재 시술용']

    df_train[cols] = df_train[cols].div(df_train[cols].sum(axis=1).replace(0, np.nan), axis=0).fillna(0)
    df_test[cols] = df_test[cols].div(df_test[cols].sum(axis=1).replace(0, np.nan), axis=0).fillna(0)

    return df_train, df_test

def 단일배아이식여부(df_train, df_val):
    df_train['단일 배아 이식 여부'] = df_train['단일 배아 이식 여부'].fillna(0)
    df_val['단일 배아 이식 여부'] = df_val['단일 배아 이식 여부'].fillna(0)
    return df_train, df_val


def 기증자정자와혼합된난자수(df_train, df_test):
    df_train["기증자 정자와 혼합된 난자 수"] = df_train["기증자 정자와 혼합된 난자 수"].fillna(2)
    df_test["기증자 정자와 혼합된 난자 수"] = df_test["기증자 정자와 혼합된 난자 수"].fillna(2)
    return df_train, df_test

def label_encoding(train, test, cols):
    encoder = LabelEncoder()
    for col in cols:
        train[col] = encoder.fit_transform(train[col])
        test[col] = encoder.transform(test[col])
    return train, test

def type_to_category(train, test, cols):
    train[cols] = train[cols].astype('category')
    test[cols] = test[cols].astype('category')
    return train, test

def impute_nan(train, test):
    cols_to_impute = [
        '임신 시도 또는 마지막 임신 경과 연수', # DI, IVF랑 관련 X
    ]
    imputer = SimpleImputer(strategy='mean')
    train[cols_to_impute] = imputer.fit_transform(train[cols_to_impute])
    test[cols_to_impute] = imputer.transform(test[cols_to_impute])

    cols_to_impute = [
        '난자 채취 경과일',
        '난자 혼합 경과일',
        '배아 이식 경과일',
        '배아 해동 경과일',

        '착상 전 유전 검사 사용 여부',
        'PGD 시술 여부',
        'PGS 시술 여부',

        ### DI only
        '착상 전 유전 진단 사용 여부',
        '총 생성 배아 수',
        '미세주입된 난자 수',
        '미세주입에서 생성된 배아 수',
        '이식된 배아 수',
        '미세주입 배아 이식 수',
        '저장된 배아 수',
        '미세주입 후 저장된 배아 수',
        '해동된 배아 수',
        '해동 난자 수',
        '수집된 신선 난자 수',
        '저장된 신선 난자 수',
        '혼합된 난자 수',
        '파트너 정자와 혼합된 난자 수',
        '기증자 정자와 혼합된 난자 수',
        '동결 배아 사용 여부',
        '신선 배아 사용 여부',
        '기증 배아 사용 여부',
        '대리모 여부',
        ### DI
    ]
    train[cols_to_impute] = train[cols_to_impute].fillna(0)
    test[cols_to_impute] = test[cols_to_impute].fillna(0)

    return train, test

def num_feature_scailing(train, test, seed=777):
    numeric_cols = train.select_dtypes(include=["number"]).columns.tolist()
    cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
    cols_to_scale = [
        col for col in numeric_cols
        if col not in cat_cols and col != '임신 성공 여부'
    ]

    arr_train = train[cols_to_scale].to_numpy()  # DataFrame -> NumPy
    arr_train = arr_train.astype(np.float32)
    arr_test = test[cols_to_scale].to_numpy()
    arr_test = arr_test.astype(np.float32)

    np.random.seed(seed)
    random.seed(seed)
    noise = (
        np.random.default_rng(0)
        .normal(0.0, 1e-5, arr_train.shape)
        .astype(arr_train.dtype)
    )
    preprocessing = QuantileTransformer(
        n_quantiles=max(min(len(train[cols_to_scale]) // 30, 1000), 10),
        output_distribution='normal',
        subsample=10**9,
    ).fit(arr_train + noise)

    # train[cols_to_scale] = preprocessing.transform(arr_train + noise)
    train[cols_to_scale] = preprocessing.transform(arr_train)
    test[cols_to_scale] = preprocessing.transform(arr_test)
    return train, test

def drop_single_value_columns(df_train, df_test):
    cols_to_drop = [col for col in df_train.columns if df_train[col].nunique() == 1]
    return df_train.drop(columns=cols_to_drop), df_test.drop(columns=cols_to_drop)





In [9]:
def all_process(train, val):
    # 기본 전처리 단계
    train, val = drop_columns(train), drop_columns(val)
    train, val = 특정시술유형(train, val)
    train, val = 시술횟수(train), 시술횟수(val)

    train, val = 단일배아이식여부(train, val)
    train, val = 배란유도유형(train, val)
    train, val = 배아생성주요이유(train, val)

    cols_to_encoding = [
        "시술 시기 코드",
        "시술 당시 나이",
        "배란 유도 유형",
        # "클리닉 내 총 시술 횟수",
        # "IVF 시술 횟수",
        # "DI 시술 횟수",
        # "총 임신 횟수",
        # "IVF 임신 횟수",
        # "DI 임신 횟수",
        # "총 출산 횟수",
        # "IVF 출산 횟수",
        # "DI 출산 횟수",
        "난자 출처",
        "정자 출처",
        "난자 기증자 나이",
        "정자 기증자 나이",
        '시술유형_통합',
    ]
    train, val = label_encoding(train, val, cols=cols_to_encoding)
    train, val = type_to_category(train, val, cols=cols_to_encoding)

    train, val = impute_nan(train, val)
    train, val = num_feature_scailing(train, val)

    train, val = drop_single_value_columns(train, val)

    return train, val

train = pd.read_csv(train_path).drop(columns=["ID"])
test = pd.read_csv(test_path).drop(columns=["ID"])

train, test = all_process(train, test)

cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]
numeric_cols = [col for col in train.columns if col not in cat_cols and col != '임신 성공 여부']

print(f'수치형 변수: {len(numeric_cols)}개 \n{numeric_cols}')
print(f'범주형 변수: {len(cat_cols)}개 \n{cat_cols}')
print(train.shape, test.shape)

/tmp/ipykernel_89/3576064510.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['배란 유도 유형'] = df_train['배란 유도 유형'].replace(mapping)
/tmp/ipykernel_89/3576064510.py:70: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test['배란 유도 유형'] = df_test['배란 유도 유형'].replace(mapping)
/tmp/ipykernel_89/3576064510.py:199: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  cat_cols = [col for col in train.columns if pd.api.types.is_catego

수치형 변수: 57개 
['임신 시도 또는 마지막 임신 경과 연수', '배란 자극 여부', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수', '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수', '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일', '시술_임신', '배아생성이유_기증용', '배아생성이유_난자 저장용', '배아생성이유_배아 저장용', '배아생성이유_현재 시술용']
범주형 변수: 8개 
['시술 시기 코드', '시술 당시 나이', '배란 유도 유형', '난자 출처', '정자 출처', '난자 기증자 나이', '정자 기증자 나이', '시술유형_통합']
(205080, 66) (51271, 65)


/tmp/ipykernel_89/2392652077.py:45: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  cat_cols = [col for col in train.columns if pd.api.types.is_categorical_dtype(train[col])]


In [ ]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU


In [ ]:
import time
start_time = time.time()



# 교차 검증 설정: seed_list를 [333] 하나만 사용, n_splits=3
seed_list = [333]
n_splits = 5

total_auc, total_acc, total_f1 = [], [], []
test_preds = []





# 교차 검증 시작
for seed in seed_list:
    set_seed(seed)
    # train, test 불러오기
    train = pd.read_csv(train_path).drop(columns=["ID"])
    test = pd.read_csv(test_path).drop(columns=["ID"])

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    auc_scores, acc_scores, f1_scores = [], [], []
    
    for fold, (train_idx, valid_idx) in enumerate(skf.split(train.drop(columns=['임신 성공 여부']), train["임신 성공 여부"])):
        # Fold 데이터 생성
        fold_train, fold_valid = train.iloc[train_idx], train.iloc[valid_idx]
        fold_train2 = fold_train.copy()
        fold_test = test.copy()  # test 데이터는 별도 사용
        
        # 전처리 
        fold_train, fold_valid = all_process(fold_train, fold_valid)
        _, fold_test = all_process(fold_train2, fold_test)

        # 범주형, 연속형 열 구분분
        categorical_cols = [col for col in fold_train.columns if pd.api.types.is_categorical_dtype(fold_train[col])]
        continuous_cols = [col for col in fold_train.columns if col not in cat_cols and col != '임신 성공 여부']


        # Wide 부분: 원-핫 인코딩 (crossed_cols 옵션도 사용할 수 있음)
        wide_cols = categorical_cols  # Wide 모델에는 범주형 변수를 원-핫 인코딩으로 처리
        crossed_cols = []  # 필요시 두 개 이상의 컬럼을 교차시켜 상호작용 feature 생성

        # WidePreprocessor (원-핫 인코딩, crossed_cols 사용 가능)
        wide_preprocessor = WidePreprocessor(wide_cols=wide_cols, crossed_cols=crossed_cols)

        # Deep 부분: 임베딩 + 연속형 변수 처리
        tab_preprocessor = TabPreprocessor(embed_cols=categorical_cols, continuous_cols=continuous_cols)

        # 전처리: 각 Fold 별로 Wide & Deep 데이터 생성
        X_wide_train = wide_preprocessor.fit_transform(fold_train)
        X_wide_valid = wide_preprocessor.transform(fold_valid)
        X_wide_test = wide_preprocessor.transform(fold_test)
        X_tab_train = tab_preprocessor.fit_transform(fold_train)
        X_tab_valid = tab_preprocessor.transform(fold_valid)
        X_tab_test = tab_preprocessor.transform(fold_test)

        # Target 값: 정수형 (0,1)
        y_train = fold_train['임신 성공 여부'].astype(int).values
        y_valid = fold_valid['임신 성공 여부'].astype(int).values

        # Wide 모델: 입력 차원은 원-핫 인코딩된 피처 수####
        wide_model = Wide(input_dim=int(X_wide_train.max().item()), pred_dim=1)


        
        # Deep 모델: FTTransformer 사용
        tab_model = FTTransformer(
            column_idx=tab_preprocessor.column_idx,  # 각 컬럼의 인덱스 정보
            cat_embed_input=tab_preprocessor.cat_embed_input,  # (컬럼명, 고유값 수, 임베딩 차원)

            continuous_cols=continuous_cols,  # 연속형 변수 리스트

            # ▶ 범주형 임베딩 관련 설정
            cat_embed_dropout=0.0,  # 범주형 임베딩 드롭아웃
            # 후보 값: 0.0 (기본), 0.1, 0.2, 0.3


            cat_embed_activation=None,  # 범주형 임베딩 활성화 함수
            # 후보 값: None (기본), 'relu', 'leaky_relu', 'tanh', 'gelu'


            # ▶ 연속형 변수 관련 설정
            cont_norm_layer="batchnorm",  # 연속형 변수 정규화 방식
            # 후보 값: None (사용 안함), 'batchnorm' (기본 추천), 'layernorm'

            embed_continuous_method="standard",  # 연속형 변수 임베딩 방식
            # 후보 값: 'standard' (기본), 'periodic', 'piecewise'

            cont_embed_dropout=0.0,  # 연속형 임베딩 드롭아웃
            # 후보 값: 0.0 (기본), 0.1, 0.2, 0.3

            cont_embed_activation='relu',  # 연속형 임베딩 활성화 함수
            # 후보 값: None (기본), 'relu', 'leaky_relu', 'tanh', 'gelu'

            quantization_setup=None,  # 'piecewise' 방식에서 구간 경계 지정
            # 후보 값: None (기본), {"age": [20, 30, 40], "income": [10000, 30000, 60000]}

            # ▶ 기타 임베딩 관련
            full_embed_dropout=False,  # 전체 임베딩을 dropout할지 여부
            # 후보 값: False (기본), True (더 강한 정규화)

            # ▶ FTTransformer 구조 설정
            input_dim=64,  # 임베딩 차원 수 (카테고리 + 연속형 임베딩 포함)
            # 후보 값: 32, 64 (기본), 128


            n_heads=8,  # Attention 헤드 수
            # 후보 값: 4, 8 (기본), 16


            n_blocks=4,  # Transformer block 수
            # 후보 값: 2 (얕은 모델), 4 (기본), 6 (깊은 모델)

            attn_dropout=0.2,  # Attention dropout
            # 후보 값: 0.0, 0.1, 0.2 (기본), 0.3



            transformer_activation="reglu",  # Transformer 내부 활성화 함수
            # 후보 값: 'relu', 'gelu', 'leaky_relu', 'tanh', 'geglu', 'reglu' (기본)

            # ▶ MLP 설정 (선택 사항)
            mlp_hidden_dims=[64, 32],  # MLP 은닉층 크기
            # 후보 값: [64, 32] (기본), [128, 64], [256, 128], None (사용 안 함)

            mlp_activation="relu",  # MLP 활성화 함수
            # 후보 값: 'relu' (기본), 'leaky_relu', 'tanh', 'gelu'

            mlp_dropout=0.1,  # MLP dropout
            # 후보 값: 0.0, 0.1 (기본), 0.3

            mlp_batchnorm=False,  # MLP에 배치 정규화 적용 여부
            # 후보 값: False (기본), True

            mlp_batchnorm_last=False,  # MLP 마지막 층에도 BN 적용 여부
            # 후보 값: False (기본), True

        )
        
        # Wide & Deep 모델 결합
        model = WideDeep(wide=wide_model, deeptabular=tab_model)
        
        # 옵티마이저 및 학습률 스케줄러 설정
        optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)  # Set your desired learning rate here
        

        # ✅ Training 데이터를 딕셔너리 형태로 생성
        X_train = {
            "X_wide": X_wide_train,
            "X_tab": X_tab_train,
            "target": y_train
        }

        # ✅ Validation 데이터를 딕셔너리로 전달 (X_val 방식 사용)
        X_val = {
            "X_wide": X_wide_valid,
            "X_tab": X_tab_valid,
            "target": y_valid
        }
        
        # EarlyStopping 콜백 설정 (patience=5, min_delta=0.001)
        early_stopping = EarlyStopping(
            monitor="val_loss", 
            patience=5, 
            min_delta=0.001, 
            verbose=0
        )
        
        # 현재 날짜/시간을 포함한 파일 이름 생성
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        model_path = f"saved_models/best_model_{timestamp}.pt"
        
        # ✅ 2. ModelCheckpoint: 최상의 모델을 자동 저장
        model_checkpoint = ModelCheckpoint(
            filepath=model_path,  # 모델 저장 경로
            monitor="val_loss",        # 감시할 지표 ('val_loss' 또는 'val_acc')
            save_best_only=True        # 가장 좋은 성능의 모델만 저장
        )
        
        # Trainer 생성: objective "binary"로 설정, 평가 지표로 Accuracy, AUROC, F1Score 사용
        trainer = Trainer(
            model=model, 
            objective="binary", 
            custom_loss_function=None,  # 기본 손실 함수 사용 (필요 시 사용자 정의 함수 가능)
            optimizers=optimizer,       # 옵티마이저 (기본값: Adam)
            initializers=None,          # 가중치 초기화 없음 (기본값)
            callbacks=[early_stopping, model_checkpoint],  # 조기 종료 (3 에포크 동안 개선 없으면 종료)
            metrics=[Accuracy()],
            verbose=0,                  # 학습 로그 출력 (기본값: 1)
            seed=seed                     # 랜덤 시드 설정 (기본값: 1)
        )
        
        # 학습: validation 데이터는 별도의 인자로 전달
        trainer.fit(
            X_train=X_train,  # ✅ Training 데이터를 딕셔너리로 
            n_epochs=100,  ####
            batch_size=1024, 
            X_val=X_val  # ✅ Validation 데이터를 딕셔너리로 전달
        )
        
        
        
        # 모델 학습 후 Validation 예측 코드:
        # 이미 확률이 계산되어 있는 컬럼을 사용합니다.
        valid_probs = trainer.predict_proba(X_wide=X_wide_valid, X_tab=X_tab_valid)[:,1] ####
        valid_pred = (valid_probs > 0.5).astype(int)

        

        # 실제 정답 
        y_valid = fold_valid['임신 성공 여부'].values.astype(int)


        
        # 평가 지표 계산: 클래스 1의 확률 사용
        fold_auc = roc_auc_score(y_valid, valid_probs)
        print(f"Seed[{seed:<3}] Fold {fold + 1} | AUC: {fold_auc:.6f}")
        
        auc_scores.append(fold_auc)
        
        total_auc.append(fold_auc)
        # Test 데이터 예측 (각 fold의 모델로 예측한 결과 저장)
        test_pred = trainer.predict_proba(X_wide=X_wide_test, X_tab=X_tab_test)[:,1]
        test_preds.append(test_pred)
    
    # Fold 별 평균 성능 출력
    avg_auc = np.mean(auc_scores)
    
    print("-" * 80)
    print(f"Seed[{seed:<3}] Average Metrics | AUC: {avg_auc:.7f}")
    print("-" * 80)

# 전체 Validation 평균 성능 출력
val_auc = np.mean(total_auc)
print("-" * 80)
print(f"Validation Average Metrics | AUC: {val_auc:.7f}")

finish_time = time.time()
total_time = finish_time - start_time 

print(total_time)

/tmp/ipykernel_89/3576064510.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['배란 유도 유형'] = df_train['배란 유도 유형'].replace(mapping)
/tmp/ipykernel_89/3576064510.py:70: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test['배란 유도 유형'] = df_test['배란 유도 유형'].replace(mapping)
/tmp/ipykernel_89/3576064510.py:199: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  cat_cols = [col for col in train.columns if pd.api.types.is_catego

Best Epoch: 5. Best val_loss: 0.49143
Model weights restored to best epoch: 10


predict: 100%|██████████| 41/41 [00:03<00:00, 12.70it/s]


Seed[333] Fold 1 | AUC: 0.734884


predict: 100%|██████████| 51/51 [00:04<00:00, 12.67it/s]
/tmp/ipykernel_89/3576064510.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_train['배란 유도 유형'] = df_train['배란 유도 유형'].replace(mapping)
/tmp/ipykernel_89/3576064510.py:70: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test['배란 유도 유형'] = df_test['배란 유도 유형'].replace(mapping)
/tmp/ipykernel_89/3576064510.py:199: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  cat_cols 

...

Best Epoch: 10. Best val_loss: 0.49094
Model weights restored to best epoch: 15


predict: 100%|██████████| 41/41 [00:03<00:00, 11.10it/s]


Seed[333] Fold 5 | AUC: 0.737963


predict: 100%|██████████| 51/51 [00:04<00:00, 12.15it/s]


--------------------------------------------------------------------------------
Seed[333] Average Metrics | AUC: 0.7370557
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Validation Average Metrics | AUC: 0.7370557
2936.452466726303


## Submission

In [ ]:
sample_submission = pd.read_csv(sample_path)

sample_submission['probability'] = np.mean(test_preds, axis=0)


sample_submission



NameError: name 'sample_path' is not defined

In [ ]:
sample_submission.to_csv(f'./Submission/FTT_{seed_list[0]}.csv', index=False)